### Project: Quaker Cookies Recipes Nutrition Data Comparison
based on ID and Title, compare ESHA column AM vs. Innit column AV 
- text match 
- may try to separate the information first by line separator /n and explode to separate rows
- Compare only: 'Calories', 'Total Fat', 'Saturated Fat', 'Cholesterol', 'Sodium', 'Carbohydrate', 'Fiber', 'Sugar', 'Protein';
- one vs. one match: calculate the % nutrient difference between two recipes of total cookies 

*Goal: Persuade dept on the future use of the new system by verifying the difference is within the acceptable gap but it consumes too much manual effort to calculate the nutrition information by hand so would like to engineer the process*

Calories 216.83 kcal
Total Fat 13.37 g
Saturated Fat 1.92 g
Cholesterol 0.0 mg
Sodium 94.83 mg
Carbohydrate 20.06 g
Fiber 4.5 g
Protein 7.35 g


In [ ]:
import pandas as pd 
data = pd.read_excel('Quaker Cookie Recipes Export.xlsx')
# keep the first 65 rows of the data
data = data.iloc[0:64, :]
data = data[['ID', 'Title', 'ESHA Food Processor Nutrition (per serving)','Nutrition Panel']]
data

,ID,Title,ESHA Food Processor Nutrition (per serving),Nutrition Panel
0,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Serves 12\nEveryday Pantry Breakfast Cookies\n...,Calories 216.83 kcal\nTotal Fat 13.37 g\n - ...
1,quaker-new-7209-en_backup,Oat Flour Chocolate Chunk Cookies,Serves 20\nOat Flour Chocolate Chunk Cookies\n...,Calories 141.56 kcal\nTotal Fat 8.05 g\n - S...
2,quaker-new-7214-en,Oat Flour Chocolate Cranberry Cookies,Serves 24\nChocolate Cranberry Oat Cookies\n\n...,Calories 92.46 kcal\nTotal Fat 5.32 g\n - Sa...
3,quaker-new-7243-en,Oat Flour Chocolate Chip Lava Cookies,Serves 10 \nChocolate Chip Lava Oat Flour Cook...,Not available.\n\nBad ingredient: sugar free d...
4,recipe-1683168170-64531baa3a55b,Quaker® 10-Minute Air Fryer Cookie,Serves 4\nQuaker 10 minute Air Fryer Breakfast...,Calories 311.98 kcal\nTotal Fat 21.69 g\n - ...
...,...,...,...,...
59,quaker-224-en,Spring-Into-Easter Cutout Cookies,Spring-Into-Easter Cutout Cookies\n\nNutrition...,Calories 75.48 kcal\nTotal Fat 3.37 g\n - Sa...
60,quaker-222-en,Double Chocolate Cookies,Double Chocolate Cookies\n\nNutrition Facts\nS...,Calories 134.54 kcal\nTotal Fat 7.24 g\n - S...
61,quaker-46-en,Sam Stephens' Oatmeal Gingerbread Cookies,NaN,Calories 149.24 kcal\nTotal Fat 5.71 g\n - S...
62,quaker-43-en,Hot Chocolate Brownie Oatmeal Cookies,Serves 72\nHot Chocolate Brownie Oatmeal Cooki...,Calories 83.29 kcal\nTotal Fat 3.73 g\n - Sa...


In [ ]:
len(data['Title'].unique()) # 64 unique recipes

64

Innit

In [7]:
# remove the '-' from the nutrition data
data['Nutrition Panel'] = data['Nutrition Panel'].str.replace('-', '')
# my data is like Calories 216.83 kcal\nTotal Fat 13.37 g\ ... i want to split it based on \n and then split it based on the first space
# split the nutrition panel into separate rows but associated with the ID and Title
#nutrition = data['Nutrition Panel'].str.split('\n', expand=True).stack().reset_index(level=1, drop=True)
nutrition = data.set_index(['ID', 'Title'])['Nutrition Panel'].str.split('\n', expand=True)
nutrition = nutrition.stack().reset_index(level=2, drop=True).reset_index()
nutrition.columns = ['ID', 'Title', 'Innit Nutrition']
#nutrition = pd.DataFrame(nutrition, columns=['Innit Nutrition'])
nutrition

,ID,Title,Innit Nutrition
0,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Calories 216.83 kcal
1,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Total Fat 13.37 g
2,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Saturated Fat 1.92 g
3,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Cholesterol 0.0 mg
4,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Sodium 94.83 mg
...,...,...,...
565,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Sodium 76.75 mg
566,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Carbohydrate 14.63 g
567,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Fiber 0.67 g
568,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Sugar 7.3 g


In [12]:
import re
# for nutrition, i want to further split it based on the first space and then create a new column for the value without the unit
target_nutrients = ['Calories', 'Total Fat', 'Saturated Fat', 'Cholesterol', 'Sodium', 'Carbohydrate', 'Fiber', 'Sugar', 'Protein']
# search for the target nutrition in the nutrition data
# if the target nutrition is in Innit Nutrition column, put the target nutrition in a new column called Nutrient and append the remaining part in Innit Nutrition into another new column called Value
# Function to split Nutrient and Value & drop the unit from the value
def extract_nutrient(row):
    for nutrient in target_nutrients:
        if nutrient in row:
            value_part = row.replace(nutrient, '').strip()
            numeric_value = re.findall(r"[-+]?\d*\.\d+|\d+", value_part)
            return pd.Series([nutrient, float(numeric_value[0]) if numeric_value else None])
    return pd.Series([None, None])

nutrition[['Nutrient', 'Innit Value']] = nutrition['Innit Nutrition'].apply(extract_nutrient)
nutrition

,ID,Title,Innit Nutrition,Nutrient,Innit Value
0,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Calories 216.83 kcal,Calories,216.83
1,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Total Fat 13.37 g,Total Fat,13.37
2,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Saturated Fat 1.92 g,Saturated Fat,1.92
3,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Cholesterol 0.0 mg,Cholesterol,0.00
4,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Sodium 94.83 mg,Sodium,94.83
...,...,...,...,...,...
565,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Sodium 76.75 mg,Sodium,76.75
566,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Carbohydrate 14.63 g,Carbohydrate,14.63
567,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Fiber 0.67 g,Fiber,0.67
568,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Sugar 7.3 g,Sugar,7.30


In [34]:
len(nutrition['Title'].unique()) # 64 unique recipes

64

EHSA

In [49]:
# for the other column, i have data like Serves 12\nEveryday Pantry Breakfast Cookies\n.. where nutrition information starts in the middle, still split it based on \n and then split it based on the first space but only keep the targeted nutrition information
# split the nutrition panel into separate rows but associated with the ID and Title
#nutrition2 = data['ESHA Food Processor Nutrition (per serving)'].str.split('\n', expand=True).stack().reset_index(level=1, drop=True)
# Replace NaNs with a placeholder (e.g., just to keep the ID/Title)
data['ESHA Food Processor Nutrition (per serving)'] = data['ESHA Food Processor Nutrition (per serving)'].fillna('')
# Then split and stack
nutrition2 = data.set_index(['ID', 'Title'])['ESHA Food Processor Nutrition (per serving)'].str.split('\n', expand=True)
nutrition2 = nutrition2.stack().reset_index(level=2, drop=True).reset_index()
nutrition2.columns = ['ID', 'Title', 'EHSA Nutrition']
#nutrition2 = pd.DataFrame(nutrition2, columns=['EHSA Nutrition'])
print(len(nutrition2['Title'].unique()))
nutrition2

64


,ID,Title,EHSA Nutrition
0,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Serves 12
1,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Everyday Pantry Breakfast Cookies
2,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,
3,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Nutrition Facts
4,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Serving size (72g)
...,...,...,...
1525,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,
1526,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Vitamin D\t0\tmcg\t\t\t0\t%\t
1527,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Calcium\t11\tmg\t\t\t0\t%\t
1528,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Iron\t0\tmg\t\t\t0\t%\t


In [50]:
# Step 1: Normalize input (replace tabs and strip)
nutrition2['EHSA Nutrition'] = nutrition2['EHSA Nutrition'].astype(str)
nutrition2['EHSA Nutrition'] = nutrition2['EHSA Nutrition'].str.replace(r'\t+', ' ', regex=True)
#nutrition2['EHSA Nutrition'] = nutrition2['EHSA Nutrition'].str.strip().replace('', pd.NA).dropna()
# Remove irrelevant rows
nutrition2 = nutrition2[~nutrition2['EHSA Nutrition'].str.contains('% Daily Value', case=False, na=False)]

target_nutrients = ['Calories', 'Total Fat', 'Saturated Fat', 'Cholesterol', 'Sodium', 'Carbohydrate', 'Fiber', 'Sugar', 'Protein']
# Step 2: Updated extract_nutrient function
def extract_nutrient(row):
    parts = row.split()

    # Mapping of aliases to standard nutrient names
    nutrient_aliases = {
        'Calories': 'Calories',
        'Total Fat': 'Total Fat',
        'Saturated Fat': 'Saturated Fat',
        'Cholesterol': 'Cholesterol',
        'Sodium': 'Sodium',
        'Dietary Fiber': 'Fiber',
        'Total Carbohydrate': 'Carbohydrate',
        'Carbohydrate': 'Carbohydrate',
        'Total Sugars': 'Sugar',
        'Protein': 'Protein'
    }

    for alias, standard_name in nutrient_aliases.items():
        if row.startswith(alias):
            for p in parts[1:]:
                if re.match(r'^-?\d+(\.\d+)?$', p):
                    return pd.Series([standard_name, float(p)])
    return pd.Series([None, None])

nutrition2[['Nutrient', 'EHSA Value']] = nutrition2['EHSA Nutrition'].apply(extract_nutrient)
print(len(nutrition2['Title'].unique()))
nutrition2

64


C:\Users\09453022\AppData\Local\Temp\ipykernel_26912\218219139.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nutrition2[['Nutrient', 'EHSA Value']] = nutrition2['EHSA Nutrition'].apply(extract_nutrient)
C:\Users\09453022\AppData\Local\Temp\ipykernel_26912\218219139.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nutrition2[['Nutrient', 'EHSA Value']] = nutrition2['EHSA Nutrition'].apply(extract_nutrient)


,ID,Title,EHSA Nutrition,Nutrient,EHSA Value
0,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Serves 12,None,NaN
1,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Everyday Pantry Breakfast Cookies,None,NaN
2,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,,None,NaN
3,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Nutrition Facts,None,NaN
4,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Serving size (72g),None,NaN
...,...,...,...,...,...
1525,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,,None,NaN
1526,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Vitamin D 0 mcg 0 %,None,NaN
1527,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Calcium 11 mg 0 %,None,NaN
1528,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Iron 0 mg 0 %,None,NaN


In [53]:
nutrition2_cleaned = nutrition2.dropna(subset=['Nutrient', 'EHSA Value'])
nutrition2_cleaned.head(2)

,ID,Title,EHSA Nutrition,Nutrient,EHSA Value
8,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Calories 260,Calories,260.0
11,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Total Fat 16 g 21 %,Total Fat,16.0


In [ ]:
len(nutrition2['Title'].unique()) # 64 unique recipes

64

Combine both systems

In [61]:
# concatenate the two dataframes into one based on the Nutrient column
nutrition_comparison = pd.merge(nutrition, nutrition2_cleaned, on=['ID','Title','Nutrient'], how='outer')
nutrition_comparison = nutrition_comparison[['ID','Title','Nutrient','Innit Value','EHSA Value']]
print(len(nutrition_comparison['Title'].unique()))
# Nutrient Percentage of Difference Calculation 
nutrition_comparison['% Difference (Per Serving)'] = ((nutrition_comparison['Innit Value'] - nutrition_comparison['EHSA Value']) / nutrition_comparison['EHSA Value'])
nutrition_comparison['% Difference (Per Serving)'] = nutrition_comparison['% Difference (Per Serving)'].round(2)
nutrition_comparison.to_excel('nutrition_comparison.xlsx', index=False)
nutrition_comparison

64


,ID,Title,Nutrient,Innit Value,EHSA Value,% Difference (Per Serving)
0,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Calories,127.82,120.0,0.07
1,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Carbohydrate,14.63,16.0,-0.09
2,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Cholesterol,23.71,5.0,3.74
3,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Fiber,0.67,1.0,-0.33
4,moresmileswitheverybite-quaker-2412-en,Quaker® Best Oatmeal Cookies,Protein,1.52,2.0,-0.24
...,...,...,...,...,...,...
574,tastyrewards-quaker-488-en,Best Oatmeal Cookies,Protein,5.28,6.0,-0.12
575,tastyrewards-quaker-488-en,Best Oatmeal Cookies,Saturated Fat,4.05,4.0,0.01
576,tastyrewards-quaker-488-en,Best Oatmeal Cookies,Sodium,323.18,340.0,-0.05
577,tastyrewards-quaker-488-en,Best Oatmeal Cookies,Sugar,26.12,27.0,-0.03


In [57]:
nutrition_comparison.head(10)

,ID,Title,Nutrient,Innit Value,EHSA Value,% Difference (Per Serving)
0,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Calories,216.83,260.0,-16.60
1,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Total Fat,13.37,16.0,-16.44
2,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Saturated Fat,1.92,2.0,-4.00
3,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Cholesterol,0.00,0.0,NaN
4,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Sodium,94.83,40.0,137.07
5,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Carbohydrate,20.06,27.0,-25.70
6,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Fiber,4.50,6.0,-25.00
7,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Sugar,7.04,9.0,-21.78
8,recipe-1694719711-65035edfc2898,Everyday Pantry Breakfast Cookies,Protein,7.35,7.0,5.00
9,quaker-new-7209-en_backup,Oat Flour Chocolate Chunk Cookies,Calories,141.56,130.0,8.89
